In [1]:
import pandas as pd

from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from surprise import accuracy

## Loading Data

In [2]:
eventDataSet = pd.read_csv('data/FilteredEvents/capstone_event_2018_07.csv')
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_08.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_09.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_10.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_11.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_12.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2019_01.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2019_02.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2019_03.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2019_04.csv'))

In [5]:
eventDataSet = eventDataSet.replace(to_replace=["Kiralık", "Satılik", "Satılık"], value = ["Kiralik", "Satilik", "Satilik"])
eventDataSet = eventDataSet[(eventDataSet.districtName.notnull())]

salesEventDataSet = eventDataSet[(eventDataSet.listingTypeId == 1)]
rentEventDataSet = eventDataSet[(eventDataSet.listingTypeId == 2)]

In [3]:
listingDataSet = pd.read_csv('data/listing/listing0.csv')

for i in range(1, 668):
    filePath = 'data/listing/listing'+str(i)+'.csv'
    listingDataSet = listingDataSet.append(pd.read_csv(filePath))

In [6]:
ds = salesEventDataSet[(salesEventDataSet.cityId == 1) & (salesEventDataSet.subPropertyType.notnull()) & (salesEventDataSet.subPropertyType == 'Daire') & (salesEventDataSet.listingId.notnull())]
ds = ds.filter(['event_date', 'event_timestamp', 'user_pseudo_id', 'event_name', 'listingId'])

ds.head()

,event_date,event_timestamp,user_pseudo_id,event_name,listingId
72,20180701,1530478077640000,bd5e859d0ca2e93dce3dbd2fe6b596f7,ListingView,1548096.0
73,20180701,1530478122579005,bd5e859d0ca2e93dce3dbd2fe6b596f7,ListingView,1615245.0
74,20180701,1530478217269000,bd5e859d0ca2e93dce3dbd2fe6b596f7,ListingView,1615167.0
75,20180701,1530478372156005,bd5e859d0ca2e93dce3dbd2fe6b596f7,ListingView,1421065.0
107,20180701,1530477950508000,39e96f9573ce4a318cc5553cc4c2f066,ListingView,1421065.0


In [12]:
bigDS = ds.set_index('listingId').join(listingDataSet.set_index('id'))
bigDS.head()

,event_date,event_timestamp,user_pseudo_id,event_name,no,listingType,type,parentType,price,currency,...,roomCount,livingRoomCount,roomSlug,size,cityId,cityName,countyId,countyName,districtId,districtName
7728.0,20190402,1554221841685024,1f936ab5b2f5fbb93ef472071eee1228,ListingView,185904.0,Sale,Daire,Konut,725000.0,TRY,...,3.0,1.0,3 + 1,150.0,1.0,İstanbul,131.0,Başakşehir,141.0,Bahçeşehir 2. Kısım
9703.0,20190419,1555701937603030,40fe5b788aa955564ffce2d2cc182f99,ListingView,65903.0,Sale,Daire,Konut,1250000.0,TRY,...,4.0,2.0,4 + 2,200.0,1.0,İstanbul,963.0,Üsküdar,997.0,Aziz Mahmut Hüdayi
21412.0,20190407,1554646137512024,03edfa3bbb7362df3d661cd5aa594cb8,ListingView,128152.0,Sale,Daire,Konut,255000.0,TRY,...,2.0,1.0,2 + 1,90.0,1.0,İstanbul,229.0,Beylikdüzü,232.0,Cumhuriyet
24909.0,20190417,1555506963250024,a088fd07e3eb73596ce5ad5c9b417ff9,ListingView,307091.0,Sale,Daire,Konut,750000.0,TRY,...,4.0,1.0,4 + 1,227.0,1.0,İstanbul,394.0,Esenyurt,45649.0,Aşık Veysel
26596.0,20180825,1535177172584108,F4A00FAF032B4168BCC3B046FFB7DBF9,ListingView,163817.0,Sale,Daire,Konut,370000.0,TRY,...,2.0,1.0,2 + 1,116.0,1.0,İstanbul,394.0,Esenyurt,403.0,Cumhuriyet


In [23]:
kkd = bigDS

In [24]:
bigDS["interest"] = bigDS.cityName + "_" + bigDS.countyName + "_" + bigDS.districtName + "_" + bigDS.roomSlug + "_" + (round(kkd.price / 50000)).astype(str)
bigDS = bigDS.filter(['user_pseudo_id', 'interest'])
bigDS['count'] = 0
ratingDS = bigDS.groupby(['user_pseudo_id', 'interest'])['count'].count().reset_index()
ratingDS.head()


,user_pseudo_id,interest,count
0,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_İçmeler_2 + 1_4.0,1
1,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_İçmeler_3 + 2_8.0,1
2,0001960a98f9d0d716e75ae49cc0df7a,İstanbul_Kartal_Cevizli_2 + 1_5.0,1
3,0001960a98f9d0d716e75ae49cc0df7a,İstanbul_Kartal_Cevizli_2 + 1_6.0,2
4,0001960a98f9d0d716e75ae49cc0df7a,İstanbul_Kartal_Cevizli_3 + 1_6.0,1


In [25]:
userSumListing = ratingDS.groupby(['user_pseudo_id'])['count'].max().reset_index()
userSumListing.columns = ['user_pseudo_id', 'maxCount']
userSumListing.head()


,user_pseudo_id,maxCount
0,00001238ac28333afc87b94a992f0319,1
1,0001960a98f9d0d716e75ae49cc0df7a,11
2,00031b94b31e128718d3c8da09b113d8,5
3,0003e4bd5debe80f65a6de85a8a63595,1
4,00079b9e760121c98236bebfe4313db6,2


In [26]:
rating = pd.concat([ratingDS.set_index('user_pseudo_id'), userSumListing.set_index('user_pseudo_id')], axis=1,
                   join='inner').reset_index()
rating['rating'] = round(3 * rating['count'] / rating['maxCount']) + 2

final = rating.filter(['user_pseudo_id', 'interest', 'rating'])
final.head()

,user_pseudo_id,interest,rating
0,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_İçmeler_2 + 1_4.0,5.0
1,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_İçmeler_3 + 2_8.0,5.0
2,0001960a98f9d0d716e75ae49cc0df7a,İstanbul_Kartal_Cevizli_2 + 1_5.0,2.0
3,0001960a98f9d0d716e75ae49cc0df7a,İstanbul_Kartal_Cevizli_2 + 1_6.0,3.0
4,0001960a98f9d0d716e75ae49cc0df7a,İstanbul_Kartal_Cevizli_3 + 1_6.0,2.0


In [27]:
reader = Reader()
data = Dataset.load_from_df(final[['user_pseudo_id', 'interest', 'rating']], reader)

In [28]:
trainset, testset = train_test_split(data, test_size=0.80)
algo = KNNBasic()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1713


1.1713324272011283

In [38]:
id = algo.trainset.to_inner_iid('İstanbul_Şişli_Mecidiyeköy_2 + 1_10.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Şişli_Mecidiyeköy_2 + 1_10.0****
İstanbul_Beylikdüzü_Cumhuriyet_2 + 1_2.0
İstanbul_Küçükçekmece_Fevzi Çakmak_2 + 1_3.0
İstanbul_Bakırköy_Ataköy 7-8-9-10. Kısım_2 + 1_26.0
İstanbul_Bağcılar_Sancaktepe_3 + 1_9.0
İstanbul_Maltepe_Zümrütevler_1 + 1_7.0
